# Time Series Forecasting with Neural DSL

This tutorial demonstrates building time series prediction models using Neural DSL.

## Overview
- Build CNN-LSTM hybrid for time series
- Handle sequential data
- Forecast future values
- Evaluate forecasting accuracy

## Setup

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

from neural.parser.parser import create_parser, ModelTransformer
from neural.code_generation.code_generator import generate_code

## Define the Time Series Model

In [ ]:
dsl_code = """
network TimeSeriesPredictor {
  input: (None, 100, 1)
  
  layers:
    Conv1D(filters=64, kernel_size=3, activation="relu", padding="same")
    BatchNormalization()
    Conv1D(filters=64, kernel_size=3, activation="relu", padding="same")
    BatchNormalization()
    MaxPooling1D(pool_size=2)
    Dropout(rate=0.2)
    
    LSTM(units=128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)
    LSTM(units=64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)
    LSTM(units=32, dropout=0.2, recurrent_dropout=0.2)
    
    Dense(units=64, activation="relu")
    BatchNormalization()
    Dropout(rate=0.3)
    Dense(units=32, activation="relu")
    BatchNormalization()
    Dropout(rate=0.3)
    Output(units=1, activation="linear")

  loss: "mse"
  optimizer: Adam(learning_rate=0.001)
  metrics: ["mae", "mse"]

  train {
    epochs: 50
    batch_size: 64
    validation_split: 0.2
  }
}
"""

with open('time_series_predictor.neural', 'w') as f:
    f.write(dsl_code)

print("Time series model defined!")

## Generate Synthetic Time Series Data

In [ ]:
def generate_time_series(n_points=10000):
    """Generate synthetic time series with trend, seasonality, and noise"""
    t = np.arange(n_points)
    
    # Trend component
    trend = 0.02 * t
    
    # Seasonal components
    seasonal_1 = 10 * np.sin(2 * np.pi * t / 365)  # Yearly
    seasonal_2 = 5 * np.sin(2 * np.pi * t / 30)    # Monthly
    
    # Noise
    noise = np.random.randn(n_points) * 2
    
    # Combine components
    series = trend + seasonal_1 + seasonal_2 + noise + 50
    
    return series

# Generate data
time_series = generate_time_series()

# Visualize
plt.figure(figsize=(15, 5))
plt.plot(time_series[:1000])
plt.title('Synthetic Time Series Data')
plt.xlabel('Time')
plt.ylabel('Value')
plt.grid(True)
plt.show()

print(f"Generated time series with {len(time_series)} points")

## Prepare Data for Training

In [ ]:
def create_sequences(data, seq_length=100):
    """Create sequences for time series prediction"""
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

# Normalize data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(time_series.reshape(-1, 1))

# Create sequences
seq_length = 100
X, y = create_sequences(scaled_data.flatten(), seq_length)

# Reshape for model input
X = X.reshape((X.shape[0], X.shape[1], 1))

# Split into train and test
split_idx = int(0.8 * len(X))
X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]

print(f"Training data shape: {X_train.shape}")
print(f"Test data shape: {X_test.shape}")
print(f"Training sequences: {len(X_train)}")
print(f"Test sequences: {len(X_test)}")

## Compile the Model

In [ ]:
!neural compile time_series_predictor.neural --backend tensorflow --output time_series_tf.py
print("Model compiled!")

## Visualize Model Architecture

In [ ]:
!neural visualize time_series_predictor.neural --format html
print("Architecture visualization generated!")

## Train the Model

In [ ]:
# Train using CLI or generated code
!neural run time_series_tf.py --backend tensorflow

## Make Predictions

In [ ]:
# Make predictions (pseudo-code)
# predictions = model.predict(X_test)
# predictions = scaler.inverse_transform(predictions.reshape(-1, 1))
# y_test_actual = scaler.inverse_transform(y_test.reshape(-1, 1))
# 
# # Visualize predictions
# plt.figure(figsize=(15, 6))
# plt.plot(y_test_actual[:200], label='Actual', linewidth=2)
# plt.plot(predictions[:200], label='Predicted', linewidth=2, alpha=0.7)
# plt.title('Time Series Predictions vs Actual')
# plt.xlabel('Time Step')
# plt.ylabel('Value')
# plt.legend()
# plt.grid(True)
# plt.show()

print("Prediction visualization code ready")

## Evaluate Model Performance

In [ ]:
# Calculate metrics
# from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
# 
# mae = mean_absolute_error(y_test_actual, predictions)
# mse = mean_squared_error(y_test_actual, predictions)
# rmse = np.sqrt(mse)
# r2 = r2_score(y_test_actual, predictions)
# 
# print(f"Mean Absolute Error: {mae:.4f}")
# print(f"Mean Squared Error: {mse:.4f}")
# print(f"Root Mean Squared Error: {rmse:.4f}")
# print(f"R² Score: {r2:.4f}")

print("Evaluation metrics template ready")

## Multi-Step Forecasting

In [ ]:
def forecast_multi_step(model, initial_sequence, n_steps, scaler):
    """Forecast multiple steps into the future"""
    predictions = []
    current_sequence = initial_sequence.copy()
    
    for _ in range(n_steps):
        # Predict next value
        next_pred = model.predict(current_sequence.reshape(1, -1, 1))[0, 0]
        predictions.append(next_pred)
        
        # Update sequence
        current_sequence = np.roll(current_sequence, -1)
        current_sequence[-1] = next_pred
    
    # Inverse transform
    predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1))
    return predictions.flatten()

# Example usage
# initial_seq = X_test[0].flatten()
# future_predictions = forecast_multi_step(model, initial_seq, 50, scaler)
# 
# plt.figure(figsize=(15, 6))
# plt.plot(range(len(initial_seq)), scaler.inverse_transform(initial_seq.reshape(-1, 1)), 
#          label='Historical', linewidth=2)
# plt.plot(range(len(initial_seq), len(initial_seq) + len(future_predictions)), 
#          future_predictions, label='Forecast', linewidth=2, linestyle='--')
# plt.title('Multi-Step Time Series Forecast')
# plt.xlabel('Time Step')
# plt.ylabel('Value')
# plt.legend()
# plt.grid(True)
# plt.show()

print("Multi-step forecasting function ready")

## Analyze Residuals

In [ ]:
# residuals = y_test_actual - predictions
# 
# fig, axes = plt.subplots(2, 2, figsize=(15, 10))
# 
# # Residual plot
# axes[0, 0].scatter(predictions, residuals, alpha=0.5)
# axes[0, 0].axhline(y=0, color='r', linestyle='--')
# axes[0, 0].set_xlabel('Predicted Values')
# axes[0, 0].set_ylabel('Residuals')
# axes[0, 0].set_title('Residual Plot')
# axes[0, 0].grid(True)
# 
# # Residual histogram
# axes[0, 1].hist(residuals, bins=50, edgecolor='black')
# axes[0, 1].set_xlabel('Residual Value')
# axes[0, 1].set_ylabel('Frequency')
# axes[0, 1].set_title('Residual Distribution')
# axes[0, 1].grid(True)
# 
# # Q-Q plot
# from scipy import stats
# stats.probplot(residuals.flatten(), dist="norm", plot=axes[1, 0])
# axes[1, 0].set_title('Q-Q Plot')
# axes[1, 0].grid(True)
# 
# # Residuals over time
# axes[1, 1].plot(residuals[:200])
# axes[1, 1].axhline(y=0, color='r', linestyle='--')
# axes[1, 1].set_xlabel('Time Step')
# axes[1, 1].set_ylabel('Residual')
# axes[1, 1].set_title('Residuals Over Time')
# axes[1, 1].grid(True)
# 
# plt.tight_layout()
# plt.show()

print("Residual analysis code ready")

## Hyperparameter Optimization

In [ ]:
!neural compile time_series_predictor.neural --backend tensorflow --hpo

## Real-World Applications

This model can be applied to:
- Stock price prediction
- Energy consumption forecasting
- Weather prediction
- Sales forecasting
- Traffic prediction
- Anomaly detection in time series

## Summary

In this tutorial, we:
1. Built a CNN-LSTM hybrid for time series
2. Generated and prepared sequential data
3. Made single and multi-step forecasts
4. Analyzed model performance and residuals

## Next Steps
- Try attention mechanisms for time series
- Implement ARIMA/SARIMA for comparison
- Use transformer models for long sequences
- Build multivariate time series models
- Implement anomaly detection